In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import joblib

In [21]:
def synthetic_data(seed=42, samples=1000):
  np.random.seed(seed);

  X = []
  y = []

  for _ in range(samples):
    label = np.random.choice([0, 1, 2])

    if label == 0:
      sequence = np.random.randint(900, 1024, size=10)
    elif label == 1:
      if np.random.rand() < 0.5:
        sequence = np.random.randint(31, 900, size=10)
      else:
        sequence = np.random.normal(loc=25, scale=1, size=10)
        sequence = np.clip(sequence, 24, 30).astype(int)
    else:
      sequence = np.random.randint(10, 31, size=10)

    X.append(sequence)
    y.append(label)

  X = np.array(X)
  y = np.array(y)
  return X,y

In [22]:
def extract_features(sequence):
  sequence = np.array(sequence)
  feature_mean = np.mean(sequence)
  feature_std = np.std(sequence)
  feature_minimum = np.min(sequence)
  feature_maximum = np.max(sequence)
  feature_range = feature_maximum - feature_minimum

  diffs = np.diff(sequence)
  mean_diff = np.mean(diffs) if len(diffs) > 0 else 0

  return [feature_mean, feature_std, feature_minimum, feature_maximum, feature_range, mean_diff]

In [23]:
if __name__ == "__main__":
  X_raw, y = synthetic_data(seed=42, samples=3000)
  print("X_raw shape:", X_raw.shape)
  print("y shape:", y.shape)

  X_features = np.array([extract_features(sequence) for sequence in  X_raw])
  print("X_features shape:", X_features.shape)

  X_train, X_test, y_train, y_test = train_test_split(
      X_features,
      y,
      test_size=0.2,
      random_state=42,
      stratify=y
  )

  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  print("Test Accuracy:", acc)
  print(classification_report(y_test, y_pred))

X_raw shape: (3000, 10)
y shape: (3000,)
X_features shape: (3000, 6)
Test Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       202
           1       1.00      1.00      1.00       209
           2       1.00      1.00      1.00       189

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [24]:
new_sequence = np.array([1024, 1024, 1024, 27, 26, 400, 500, 99, 29])
new_features = np.array(extract_features(new_sequence)).reshape(1, -1)
prediction = model.predict(new_features)
predicted_class = prediction[0]

joblib.dump(model, "model.pkl")
files.download("model.pkl")

print("New sequence:", new_sequence, "| Predicted class:", predicted_class)
if predicted_class == 0:
  print(">>> No Fire")
elif predicted_class == 1:
  print(">>> Suspicious Activity")
else:
  print(">>> Fire")

New sequence: [1024 1024 1024   27   26  400  500   99   29] | Predicted class: 1
>>> Suspicious Activity
